# get_frequently_mutated Final 

In [8]:
import cptac
#en = cptac.Endometrial()
co = cptac.Colon()
#ov = cptac.Ovarian()
rc = cptac.RenalCcrcc()

import pandas as pd
import numpy as np

In [24]:
def get_frequently_mutated(cancer_object, cutoff = 0.1):  
    """
    Takes a cancer object and find the frequently 
    mutated genes (in the tumor samples) compared to the cutoff.

    Parameters:
    cancer_object (object): cancer type from cptac module 
    cutoff (float): used as a comparison to determine the 
                    status of gene mutation frequency

    Returns:
    freq_mutated_df (pd.DataFrame): DataFrame of frequently 
        mutated genes passing the cutoff. Columns contain the 
        fractions of total unique mutations,missence type 
        mutations, and truncation type mutations per gene.
    
    The Missence_Mut column includes: 
        In_Frame_Del, In_Frame_Ins, Missense_Mutation
   
    The Truncation_Mut column includes: 
        Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, 
        Nonsense_Mutation, Nonstop_Mutation
        
    These columns count multiple mutations of one gene in the 
    same sample, so fractions in the last two columns may 
    exceed the Unique_Samples_Mut column which only counts if 
    the gene was mutated once per sample."""    
    
    # Get total tumors/patients
    omics_and_mutations = cancer_object.join_omics_to_mutations(
        mutations_genes = 'TP53', omics_df_name = 'proteomics', omics_genes = 'TP53')
    tumors = omics_and_mutations.loc[omics_and_mutations['Sample_Status'] == 'Tumor'] 
    total_tumor_samples = len(tumors)
    
    # Get mutations data frame
    somatic_mutations = cancer_object.get_mutations() 
    origin_df = somatic_mutations.reset_index() #prepare to count unique samples
    
    # Drop silent mutations for Ovarian and RenalCcrcc dataset 
    if 'Silent' in origin_df['Mutation'].unique():
         origin_df = origin_df.loc[origin_df['Mutation'] != 'Silent']
         print("dropped Silent mutations")
        
    # Create two categories in Mutation column - 'M': Missence, 'T': Truncation
    if cancer_object.get_cancer_type() == 'colon':
        missence_truncation_groups = {'frameshift substitution': 'T', 
            'frameshift deletion': 'T', 'frameshift insertion': 'T', 
            'stopgain': 'T', 'stoploss': 'T', 'nonsynonymous SNV': 'M',
            'nonframeshift insertion': 'M','nonframeshift deletion': 'M', 
            'nonframeshift substitution': 'M'}
    else: 
        missence_truncation_groups = {'In_Frame_Del': 'M', 'In_Frame_Ins': 'M',
            'Missense_Mutation': 'M', 'Frame_Shift_Del': 'T','Nonsense_Mutation': 'T', 
            'Splice_Site': 'T', 'Frame_Shift_Ins': 'T','Nonstop_Mutation':'T'}
    mutations_replaced_M_T = origin_df.replace(missence_truncation_groups)
    
    # Find frequently mutated genes (total fraction > cutoff)
    # Step 1 - group by gene and count unique samples
    count_mutations = origin_df.groupby(['Gene']).nunique()
    # Step 2 - format
    count_mutations = count_mutations.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
    count_mutations = count_mutations.drop(['Gene', 'Mutation','Location'], axis = 1)
    # Step 3 - filter using the cutoff and create fraction
    fraction_mutated = count_mutations.apply(lambda x: x / total_tumor_samples)
    fraction_greater_than_cutoff = fraction_mutated.where(lambda x: x > cutoff) #na used when false
    filtered_gene_df = fraction_greater_than_cutoff.dropna()
    
    # Create and join Missence column (following similar steps as seen above)
    miss = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'M']
    count_miss = miss.groupby(['Gene']).nunique()
    missence_df = count_miss.rename(columns={"Sample_ID": "Missence_Mut"})
    missence_df = missence_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    fraction_missence = missence_df.apply(lambda x: x / total_tumor_samples)
    freq_mutated_df = filtered_gene_df.join(fraction_missence, how='left').fillna(0)
    
    
    # Create and join Truncation column (following similar steps as seen above)
    trunc = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'T']
    count_trunc = trunc.groupby(['Gene']).nunique()
    truncation_df = count_trunc.rename(columns={"Sample_ID": "Truncation_Mut"})
    truncation_df = truncation_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    fraction_truncation = truncation_df.apply(lambda x: x / total_tumor_samples)
    freq_mutated_df = freq_mutated_df.join(fraction_truncation, how='left').fillna(0)

    return freq_mutated_df

In [25]:
df = get_frequently_mutated(rc, .1)
df.reset_index()

dropped Silent mutations


,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,BAP1,0.154545,0.063636,0.090909
1,KDM5C,0.172727,0.036364,0.145455
2,PBRM1,0.400000,0.072727,0.336364
3,SETD2,0.136364,0.018182,0.118182
4,TTN,0.118182,0.090909,0.036364
5,VHL,0.745455,0.300000,0.445455


In [45]:
df = get_frequently_mutated(en, .2)
df

,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
Gene,,,
ARID1A,0.452632,0.136842,0.400000
CTCF,0.284211,0.094737,0.242105
CTNNB1,0.305263,0.305263,0.000000
KMT2B,0.242105,0.115789,0.126316
KRAS,0.326316,0.326316,0.000000
PIK3CA,0.494737,0.484211,0.010526
PIK3R1,0.389474,0.231579,0.189474
PTEN,0.789474,0.463158,0.568421
TP53,0.221053,0.157895,0.073684


In [46]:
df = get_frequently_mutated(ov, .2)
df

,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
Gene,,,
MUC4,0.325301,0.313253,0.012048
TP53,0.927711,0.602410,0.325301
TTN,0.301205,0.265060,0.084337


In [7]:
#test results
r = cptac.RenalCcrcc()
m = r.get_mutations()

#m = m.loc[m['Mutation'] != 'Silent']
gene = 'FSIP2'
g = m.loc[m['Gene'] == gene]
#print(g)
print(len(g.index.unique()))

r = g.groupby('Mutation')

r.groups


5                                   


{'Frame_Shift_Del': Index(['S117'], dtype='object', name='Sample_ID'),
 'Frame_Shift_Ins': Index(['S039'], dtype='object', name='Sample_ID'),
 'Missense_Mutation': Index(['S004', 'S108', 'S131'], dtype='object', name='Sample_ID')}